In [3]:
import urllib.request, urllib.parse, urllib.error
import ssl
from bs4 import BeautifulSoup as bs
import sqlite3

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# setup sqlite database
connectDB = sqlite3.connect('news.sqlite')
cur = connectDB.cursor()


In [10]:
# scraping parameters
startyear = 2018
endyear = 2018
startmonth = 3
endmonth = 12
startday = 1
endday = 31

# looping through years, months, days
print("begin...")
for year in range(startyear,endyear+1):
  for month in range(startmonth,endmonth+1):
    for day in range(startday,endday+1):
      page = 1
      redirected = False

      # looping through every pages in index
      while True:

        # skip to next loop if page is redirected
        if redirected == True:
          break
        
        # trying until the request replied
        while True:
          try:
            concatedURL = 'https://www.jpnn.com/indeks?id=276&d='+str(day)+'&m='+str(month)+'&y='+str(year)+'&tab=all&page='+str(page)
            reqIndex = urllib.request.Request(concatedURL, headers={'User-Agent': 'Mozilla/5.0'})
            indeks = urllib.request.urlopen(reqIndex, context=ctx)
          except:
            print("retrying connection...")
            time.sleep(1)
            continue
          break
        
        # breaking the loop if redirected and pass redirected parameter to break parent loop
        if concatedURL != indeks.geturl(): 
          print(concatedURL+" is redirected to "+indeks.geturl())
          redirected = True
          break
        
        # selecting links in indeks
        indeks = indeks.read()
        parsedIndex = bs(indeks, 'html.parser')
        linkSelector = '#content-utama > div.kolom-kiri > div.border-box > div > ul > li > div.content-description > div > h2 > a'
        links = parsedIndex.select(linkSelector)

        # iterate every links
        for link in links:
          title = link.get_text()
          contentURL = urllib.parse.quote(link['href'], safe="http://")

          # same tricks as earlier request
          while True:
            try:
              reqContent = urllib.request.Request(contentURL, headers={'User-Agent': 'Mozilla/5.0'}) 
              content = urllib.request.urlopen(reqContent, context=ctx).read()
            except:
              print("retrying connection...")
              time.sleep(1)
              continue
            break
          
          # obtaining location data
          parsedContent = bs(content, 'html.parser')
          locationSelector = '#content-utama > div.kolom-kiri > div.border-box > div > div.content-scroll > div.page-content > div > p > a'
          locationData = parsedContent.select(locationSelector)

          # if not available, assign "None" value
          try: 
            location = locationData[0].contents[0]
          except: 
            location = "None"
            pass
          
          # obtaining first page content
          fpSelector = '#content-utama > div.kolom-kiri > div.border-box > div > div.content-scroll > div.page-content > div > p'
          fpData = parsedContent.select(fpSelector)

          # if no content, use None
          try:
            fp = fpData[0].get_text()
          except:
            fp = None

          date = str(year)+'-'+str(month)+'-'+str(day)
          link = link['href']
          
          # insert new row into database
          cur.execute('INSERT INTO Source (title, firstparagraph, date, location, link) VALUES (?, ?, ?, ?, ?)', (title, fp, date, location, link))
          connectDB.commit()

        print(str(day)+'/'+str(month)+'/'+str(year)+" page "+str(page)+' was parsed')

        # find whether there is more pages
        nextButton = parsedIndex.find_all(rel='next') 
        if nextButton == []:
          break
        else:
          page += 1
                

OperationalError: database is locked